In [1]:
import torch
import pandas as pd
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from tqdm import tqdm
import evaluate


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = XLNetTokenizer.from_pretrained('XLNet-large-cased')
model = XLNetForSequenceClassification.from_pretrained('XLNet-large-cased', num_labels=3)

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at XLNet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('data/forex_news.csv')
X = df['news'].tolist()
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}

In [4]:
labels=df['labels'].map(label_mapping).tolist()

In [5]:
# Load evaluation metrics (accuracy & F1-score)
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [6]:
input_ids = []
attention_masks = []
texts=X
for text in texts:
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=46, return_tensors="pt")
    input_ids.append(encoding.input_ids)
    attention_masks.append(encoding.attention_mask)
input_ids = torch.cat(input_ids, dim=0).to('cuda')
attention_masks = torch.cat(attention_masks, dim=0).to('cuda')
model.to('cuda')
labels = torch.tensor(labels).to('cuda')
# Split the data into train, validation, and test sets
train_inputs, test_inputs, train_masks, test_masks, train_labels, test_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42
)

In [7]:
# Create DataLoader for training and testing
batch_size = 16
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [8]:
# Define the optimizer and loss function
weight_decay=0.01
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5,weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)
loss_fn = torch.nn.CrossEntropyLoss()
# Training loop


for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids, attention_mask, labels = batch
        labels = labels.long()
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits 
        loss = loss_fn(logits, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation loop
    model.eval()
    nb_eval_steps = 0
    true_labels = []
    predicted_labels = []
    for batch in tqdm(test_dataloader, desc="Validation"):
        input_ids, attention_mask, labels = batch
        labels = labels.long()
        with torch.no_grad():
            logits = model(input_ids, attention_mask=attention_mask).logits
        predicted_labels2 = torch.argmax(logits, dim=1)
        nb_eval_steps += 1
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted_labels2.cpu().numpy())
    f1 = f1_metric.compute(predictions=predicted_labels, references=true_labels, average="weighted")["f1"]
    accuracy = accuracy_metric.compute(predictions=predicted_labels, references=true_labels)["accuracy"]
    print(f"Epoch {epoch + 1}/{epochs}: Avg. Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {accuracy:.4f}, f1: {f1:.4f}")



C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:37<00:00,  1.74it/s]


Epoch 1/10: Avg. Training Loss: 1.0872, Validation Accuracy: 0.4864, f1: 0.3184


Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:37<00:00,  1.75it/s]


Epoch 2/10: Avg. Training Loss: 0.9382, Validation Accuracy: 0.5039, f1: 0.4620


Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:41<00:00,  1.57it/s]


...

Epoch 8/10: Avg. Training Loss: 0.1945, Validation Accuracy: 0.7868, f1: 0.7910


Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:35<00:00,  1.84it/s]


Epoch 9/10: Avg. Training Loss: 0.1560, Validation Accuracy: 0.8091, f1: 0.8118


Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:36<00:00,  1.79it/s]


Epoch 10/10: Avg. Training Loss: 0.1110, Validation Accuracy: 0.8101, f1: 0.8126


In [9]:
model.save_pretrained( "C:\\Users\\Admin\\forex-sentiment-analysis\\models\\forex-analysis-model")